# Creating puzzle grids

In [2]:
import nltk

## Wordnet

This application will be built in top of WordNet, which will also be cited on the main page itself:

Princeton University "About WordNet." [WordNet](https://wordnet.princeton.edu/). Princeton University. 2010. 

In [3]:
from nltk.corpus import wordnet as wn

In [4]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /home/hector/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/hector/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [5]:
base_dog = wn.synsets('dog')[0]

In [6]:
def synset_word(synset: "Synset") -> str:
    return synset.name().split(".")[0]

In [7]:
from enum import Enum
from dataclasses import dataclass
from typing import *

In [8]:
class LinkType(Enum):
    Synset = 1
    Hyponym = 2
    Hypernym = 3
    Homonym = 4
    Root = 5
    Synonym = 6

In [9]:
@dataclass
class Link:
    synset : "Synset"
    type : LinkType

In [10]:
def one_step(origin: "Synset") -> List[Link]:
    hypernyms = map(lambda s: Link(s, LinkType.Hypernym), origin.hypernyms())
    hyponyms = map(lambda s: Link(s, LinkType.Hyponym), origin.hyponyms())
    homonyms = map(lambda s: Link(s, LinkType.Homonym), wn.synsets(synset_word(origin)))
    return list(hypernyms) + list(hyponyms) + list(homonyms)

In [11]:
#one_step(base_dog)

In [12]:
#Spoof class for type checking
class Synset:
    pass

In [13]:
@dataclass
class Path:
    origin: "Synset"
    links: List[Link]

    @property
    def synset(self):
        assert len(self.links) > 0
        return self.links[-1].synset


In [14]:
def ball(origin_synset: "Synset", distance: int):
    origin_points = [Path(origin_synset, [Link(origin_synset, LinkType.Root)])]
    synsets_so_far: List[Synset] = [path.synset for path in origin_points]
    paths_so_far: List[Path] = [point for point in origin_points]
    outer_shell: List[Path] = [point for point in origin_points]
    for i in range(distance):
        next_outer_shell: List[Path] = []
        for point in outer_shell:
            novel_steps: List[Link] = [step for step in one_step(point.synset)
                                       if step.synset not in synsets_so_far]
            accepted_points: List[Path] = [Path(origin_synset, [link for link in point.links] + [p])
                                           for p in novel_steps]
            for new_point in accepted_points:
                paths_so_far.append(new_point)
                synsets_so_far.append(new_point.synset)
                next_outer_shell.append(new_point)
        outer_shell = [point for point in next_outer_shell]
    return paths_so_far

In [15]:
def spheres(origin_word: str, distance: int) -> List[List[Path]]:
    all = ball(origin_word, distance)
    spheres = []
    for i in range(1, distance+1):
        spheres.append([path for path in all if len(path.links) == i])
    return spheres

In [16]:
import random
def rfrom(_list):
    return _list[int(random.random() * len(_list))]

In [17]:
def random_synset_for(word: str):
    synsets = wn.synsets(word)
    return rfrom(synsets)

In [18]:
spheres(random_synset_for("hat"), 2)

[[Path(origin=Synset('hat.v.02'), links=[Link(synset=Synset('hat.v.02'), type=<LinkType.Root: 5>)])],
 [Path(origin=Synset('hat.v.02'), links=[Link(synset=Synset('hat.v.02'), type=<LinkType.Root: 5>), Link(synset=Synset('supply.v.01'), type=<LinkType.Hypernym: 3>)]),
  Path(origin=Synset('hat.v.02'), links=[Link(synset=Synset('hat.v.02'), type=<LinkType.Root: 5>), Link(synset=Synset('hat.n.01'), type=<LinkType.Homonym: 4>)]),
  Path(origin=Synset('hat.v.02'), links=[Link(synset=Synset('hat.v.02'), type=<LinkType.Root: 5>), Link(synset=Synset('hat.n.02'), type=<LinkType.Homonym: 4>)]),
  Path(origin=Synset('hat.v.02'), links=[Link(synset=Synset('hat.v.02'), type=<LinkType.Root: 5>), Link(synset=Synset('hat.v.01'), type=<LinkType.Homonym: 4>)])]]

# Making games out of this

## Center word

Idea: Select the correct center word from a list of nine words.

First we read in the list of common words. The correct answer should be inside this list.

In [19]:
import re
def read_moby():
    path="moby-common.txt"
    restriction = r"(\W|\.|-|\d|[A-Z])"
    with open(path, "r") as file:
        words_with_linebreak = file.readlines()
    words_with_bad_characters = [word.replace("\n","") for word in words_with_linebreak]
    no_bad_characters = [word for word in words_with_bad_characters if re.search(restriction, word) is None]
    right_length = [word for word in no_bad_characters if len(word) in range(3,8)]
    return right_length

common_words = read_moby()

In [20]:
def assess_synset_get_spheres(synset: "Synset"):
    try:
        word_spheres = spheres(synset,3)
        return word_spheres
    except Exception as e:
        print(f"Rejecting {synset}: uncaught exception {e}")
        return None

What do we display to the player?

In [21]:
@dataclass
class CWPlayerOption:
    word: str
    path: Path
    description: str

In [22]:
def define(synset: "Synset"):
    return f"{synset_word(synset)}: {synset.definition()}"

Don't accept qualified synsets (ones with spaces)

In [23]:
def accept_synset(synset: "Synset") -> bool:
    word = synset_word(synset)
    if "_" in word:
        return False

    return True

In [24]:
def accept_path(path: Path) -> bool:
    return all(map(lambda link: accept_synset(link.synset), path.links))

Make the chain human readable

In [25]:
def path_to_description(path: Path):
    base = ""
    connectors = {
        LinkType.Synset: "",
        LinkType.Homonym: " -(alternate meaning)-> ",
        LinkType.Hypernym: " -(an example of)-> ",
        LinkType.Hyponym: " -(an example being)-> ",
        LinkType.Root: "",
        LinkType.Synonym: " -(synonymous with)-> ",
    }
    for link in path.links:
        base += connectors[link.type]
        base += define(link.synset)
    return base


We want the words to be distinct enough to avoid, e.g. "live" and "alive" being in the grid together.
Use fuzzy matching to discriminate against words similar to words already accepted.

In [26]:
import fuzzywuzzy
import fuzzywuzzy.fuzz

/mnt/f/github/personal/spiderweb/.venv/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


The grid needs to have a unique solution,
so that all other words are within distance 2 of the target,
and so that the words in the grid are path-connected using just those words.

In [33]:
def build_cw_game(synset: "Synset"):
    word_spheres = assess_synset_get_spheres(synset)
    if word_spheres is None:
        return None
    distant: List[Path] = [
        path for path in word_spheres[2] if accept_path(path)]
    near: List[Path] = [path for path in word_spheres[1] if accept_path(path)]
    random.shuffle(distant)
    random.shuffle(near)

    origin = Path(synset, [Link(synset, LinkType.Root)])

    chosen_far: List[CWPlayerOption] = []
    chosen_near: List[CWPlayerOption] = []
    words_used: List[str] = [synset_word(synset)]

    allowed_ratio = 80

    def similar_word_used(word: str):
        ratios = [fuzzywuzzy.fuzz.ratio(word, existing_word)
                  for existing_word in words_used]
        return max(ratios) > allowed_ratio

    while len(chosen_far + chosen_near) < 7 and len(distant) > 0:
        path = distant.pop()
        word = synset_word(path.synset)
        bridge_word = synset_word(path.links[1].synset)
        bridge_path = Path(path.origin, path.links[:-1])

        ratio = fuzzywuzzy.fuzz.ratio(word, bridge_word)

        if not ratio > allowed_ratio:
            near_already_exists = path_to_description(bridge_path) in [
                path_to_description(option.path) for option in chosen_near]

            if not similar_word_used(word):
                if near_already_exists:
                    chosen_far.append(CWPlayerOption(
                        word, path, path_to_description(path)))
                    words_used.append(word)
                else:
                    if not similar_word_used(bridge_word):
                        if not word == bridge_word:
                            chosen_far.append(CWPlayerOption(
                                word, path, path_to_description(path)))
                            chosen_near.append(CWPlayerOption(
                                bridge_word, bridge_path, path_to_description(bridge_path)))
                            words_used.append(word)
                            words_used.append(bridge_word)

    if len(chosen_near) < 2:
        # Need a path of length 5 to have a hope of determining the center
        return None

    while len(chosen_far + chosen_near) < 8 and len(near) > 0:
        path = near.pop()
        word = synset_word(path.synset)
        if word not in words_used:
            chosen_near.append(CWPlayerOption(
                word, path, path_to_description(path)))
            words_used.append(word)

    # Need unique right answer
    # Each wrong answer needs at least one word that is distance > 2 from it
    for answer in chosen_far + chosen_near:
        for potential_target_synset in wn.synsets(answer.word):
            ball_around_option = assess_synset_get_spheres(potential_target_synset)
            words_within_two = [synset_word(
                path.synset) for layer in ball_around_option for path in layer]
            if all([word in words_within_two for word in words_used]):
                return None

    return chosen_far + chosen_near + [CWPlayerOption(synset_word(synset), origin, path_to_description(origin))]


A game is really just 9 options, one of which is correct.

In [34]:
build_cw_game(random_synset_for("shape"))

Here's what the player will see (randomised, so may need to run a few times)

In [35]:
game = build_cw_game(random_synset_for("shape"))
if game is not None:
    for option in game:
        print(option.word)

Now let's sample a large number of acceptable target words, and build webs around those.

In [36]:
selected_common_words = [rfrom(common_words) for _ in range(100000)]
cw_games : List[List[CWPlayerOption]] = []

Approximately 10% of starting words lead to a valid game.

In [37]:
def get_game_or_none(word: str) -> Optional["game"]:
    try:
        for synset in wn.synsets(word):
            if synset_word(synset) == word:
                game = build_cw_game(synset)
                if game is not None:
                    if len(game) == 9:
                        return game
    except Exception as e:
        return None

In [39]:
from tqdm import tqdm

In [44]:
for word in tqdm(selected_common_words[1000:2000]):
    game = get_game_or_none(word)
    if game is not None:
        cw_games.append(game)


100%|██████████| 1000/1000 [47:34<00:00,  2.85s/it] 


In [46]:
len(cw_games)

338

In [47]:
def cw_mock_board(list_of_options: List[CWPlayerOption]):
    for option in list_of_options:
        print(option.word)

In [48]:
cw_mock_board(cw_games[11])

communicate
frown
wince
squint
fleer
grimace
smirk
grin
smile


This all needs to be stored as JSON so it can be read by the javascript frontend.

In [49]:
def cw_compress_option(option: CWPlayerOption)-> Dict[str, Any]:
    return {"s": 3-len(option.path.links), "w": option.word, "d": option.description}

In [50]:
def cw_compress_game(game) -> List[Dict[str, Any]]:
    return list(map(cw_compress_option, game))   

In [51]:
import json

Some basic checks are made:
 - profanity
 - enough distance-2 words
 - enough words are in our list of common words
 - words aren't too short (the letter "c" was once  a suggested word)

In [52]:
def cw_restrict_games(games_list: List["game"]) -> List["game"]:
    # The following words came up in tests
    # And I want to exclude games that use them

    blocked_words = ["derogatory", "insult", "insulting",
                     "whore", "pimp", "fuck", "obscene", "shit"]

    def pass_option(option: CWPlayerOption) -> bool:
        # Simple word filter
        words_used = re.split("\W", option.description)
        return all([word not in words_used for word in blocked_words])

    def pass_game(game: List[CWPlayerOption]) -> bool:
        # Check the shortest word is at least 3 letters
        shortest = min([len(option.word) for option in game])
        if shortest < 3:
            return False
        # Check there is at least two paths of length 2
        furthest = sum([len(option.path.links) == 3 for option in game])
        if furthest < 1:
            return False
        # Check at least 3 words are in the list of common words
        count_common = sum(
            [1 for option in game if option.word in common_words])
        if count_common < 3:
            return False
        # Check the individual options are allowed
        return all([pass_option(option) for option in game])
    return [game for game in games_list if pass_game(game)]


How many does that leave us with?

In [53]:
len(cw_restrict_games(cw_games))

324

Write to JSON

In [54]:
with open("webs.json", "w") as file:
    json.dump(list(map(cw_compress_game, cw_restrict_games(cw_games))), file)